## Definicje funkcji

In [128]:
def FIFO(input_set, num_frames): #input_set - lista
    output_string = ""
    frames = num_frames * [None]
    
    for element in input_set:
        
        if element in frames:
            output_string += "---\n"
            continue
        
        #poniższa pętla szuka wolnego miejsca w ramkach
        if None in frames:
            for idx, frame in enumerate(frames): #zwraca element frame i indeks(do modyfikacji ramek)
                if frame is None:
                    frames[idx] = element
                    break
        
        else:
            del frames[0]
            frames.append(element)
        output_string += str(frames) + "\n"
        
    return output_string


def OPT(input_set, num_frames):
    output_string = ""
    frames = num_frames * [None]
    
    for el_idx, element in enumerate(input_set):
        
        if element in frames:
            output_string += "---\n"
            continue
            
        if None in frames:
            for idx, frame in enumerate(frames):
                if frame is None:
                    frames[idx] = element
                    break
                    
        else:
            subset_to_check = input_set[el_idx+1:]
            
            oldest_idx = -1
            oldest_value = -1
            for frame in frames:
                if frame is not None:
                    try:
                        current_idx = subset_to_check.index(frame) #najwczesniejszy indeks sprawdzanej wartosci z ramki
                    except ValueError:
                        oldest_value = frame #sprawdzana wartoć nie istnieje w zbiorze sprawdzanych wartości, dlatego f-cja subset.. wyrzuca exception
                        break
                    else: #jeżeli jest wartość w zbiorze sprawdzanych wartości
                        if oldest_idx < current_idx:
                            oldest_idx = current_idx
                            oldest_value = subset_to_check[oldest_idx]
            
            frames[frames.index(oldest_value)] = element #sprawdzany jest idx wartości, która ma być nadpisana przez nowy proces
        
        output_string += str(frames) + "\n"
        
    return output_string

# Implementacja LRU za pomocą stosu
def LRU(input_set, num_frames):
    output_string = ""
    frames = num_frames * [None]
    
    # Pętla iterująca po kolejnych zadaniach umieszczanych w stronach pamięci
    for el_idx, element in enumerate(input_set):
        # Jeżeli proces jest już w pamięci, zostaw go tam i przejdź do następnego elementu
        if element in frames:
            # Usuń istniejące zadanie z ramek pamięci
            del frames[frames.index(element)] 
            
            # Umieść nowe zadanie na szczycie stosu (pierwsza ramka pamięci)
            frames = [element] + frames
            
            output_string += str(frames) + "\n"
            continue
            
        # Jeżeli są jeszcze puste ramki, wypełniaj je zadaniami (None są na górze stosu)
        if None in frames:
            for idx, frame in reversed(list(enumerate(frames))):
                if frame is None:
                    frames[idx] = element
                    break
        else:
            # Przyszło nowe zadanie i nie ma już pustych ramek do wypełnienia
            
            # Usuń najdawniej używane zadanie (umieszczone na dnie stosu, stąd ostatni element z ramek usuwamy)
            del frames[-1]
            
            # Nowe zadanie na górze stosu
            frames = [element] + frames
            
        output_string += str(frames) + "\n"
    
    return output_string

def SCA(input_set, num_frames):
    output_string = ""
    frames = num_frames * [None]
    all_second_chance = False
    
    # Lista bitów odniesienia
    give_second_chance = [False] * num_frames
    
    for el_idx, element in enumerate(input_set):
         
        # Jeżeli element jest już w ramkach, to daj mu drugą szansę (ustawiając bit drugiej szansy na True)
        if element in frames:
            give_second_chance[frames.index(element)] = True            
            output_string += str(frames) + " | " + str(give_second_chance) + "\n"       
            continue
            
        # Jeżeli są jeszcze puste ramki, wypełniaj je zadaniami (None są na górze stosu)
        if None in frames:
            for idx, frame in reversed(list(enumerate(frames))):
                if frame is None:
                    frames[idx] = element
                    break
        else:
            # Jeżeli najstarsza ramka (dolna) jest False (nie ma danej drugiej szansy), to usuń ją i na szczycie FIFO daj nowe zadanie
            if give_second_chance[-1] is False:
                del frames[-1]
                frames = [element] + frames
            else:
                # Jeżeli najstarsza wg FIFO ramka ma drugą szansę, to usuń jej drugą szansę i szukaj dalej wolnego miejsca do zamiany
                give_second_chance[-1] = False
                
                # Sprawdzimy kolejne elementy (od najstarszego wg FIFO) - lecimy od tyłu listy, stąd ujemne indeksy
                idx_el_to_change = -2
                
                while True:
                    
                    # Ta sytuacja wystąpi tylko wtedy, gdy wszystkie zadania w ramkach mają drugą szansę. Wtedy przerywamy pętlę i wykona się
                    # kod od razu poza nią (# 2nd Chance)
                    if np.abs(idx_el_to_change) > num_frames:
                        all_second_chance = True
                        break
                        
                    if give_second_chance[idx_el_to_change] is False:
                        del frames[idx_el_to_change]
                        frames = [element] + frames
                        break
                    else:
                        give_second_chance[idx_el_to_change] = False
                        idx_el_to_change -= 1
                
                # 2nd Chance
                if all_second_chance is True:
                    del frames[-1]
                    frames = [element] + frames
                    
        output_string += str(frames) + " | " + str(give_second_chance) + "\n"       
    
    return output_string

In [265]:
import numpy as np

def load_reference_set_from_file(filename):
    with open(filename, "r") as file:
        data = file.read()
        data = data.replace("\n","").split(" ")
        data = list(map(int, data))
        
    return data

def load_data_from_user():
    data = input("Prosze podac ciag odniesien (oddzielonych spacja): ")
    data = data.replace("\n", "").split(" ")
    data = list(map(int, data))
    
    num_frames = input("Prosze podac liczbe ramek: ")
    num_frames = num_frames.replace("\n", "")
    num_frames = int(num_frames)
    
    return data, num_frames

def save_to_file(output, filename):
    with open(filename, "w") as file:
        file.write(output) 
    
def generate_reference_set(set_size):
    return np.random.randint(0,10,size=set_size).tolist()



class Process():
    def __init__(self, number, priority, enter, duration):
        self.number = number
        self.priority = priority
        self.enter = enter
        self.duration = duration
        
        self.remaining_time = duration
#         self.wait_time = 0
        
        self.being_executed = False
        
    def execute(self):
        self.remaining_time -= 1
                
#     def wait(self):
#         self.wait_time += 1
        
        
def convert_to_objects(data_from_file):
    tasks = []
    
    for process in data_from_file:
        task = Process(process[0], process[1], process[2], process[3])
        tasks.append(task)
        
    return tasks

def load_tasks_from_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
        del lines[-1]
        
        data = []
        for idx, line in enumerate(lines):
            lines[idx] = line.replace("\n", "")
            data.append(list(map(int, lines[idx].split(" "))))
            
    tasks = convert_to_objects(data)
    return tasks

## Uruchomienie funkcji

In [462]:
tasks = load_tasks_from_file("dane_wejsciowe.txt") # Wczytuje plik i od razu przedstawia go w postaci obiektów typu Process

In [463]:
def FCFS(list_of_processes):
    
    time = 0
    
    # Ustaw procesy w kolejności wykonywania (wg czasu przyjścia)
    tasks = sorted(list_of_processes, key=lambda x: x.enter)    
    wait_time = [0] * len(tasks)
    
    while len(tasks) > 0:
                
        tasks[0].being_executed = True
        while tasks[0].remaining_time > 0:
            print(str(time + 1) + " | " + str(tasks[0].number))
            tasks[0].execute()
            
            time += 1
            
            for task in tasks:
                if not task.being_executed and task.enter < time:
                    wait_time[task.number - 1] += 1  
        
        del tasks[0]
        
    return wait_time
        
# SJF bez wywłaszczenia
def SJF_nonpreemptive(list_of_processes):
    
    time = 0
    
    # Ustaw procesy wg czasu przyjścia
    tasks = sorted(list_of_processes, key=lambda x: x.enter)
    wait_time = [0] * len(tasks)
    
    while len(tasks) > 0:
        
        ready_to_be_executed = []
        # Stwórz listę zadań, które mogą być już wykonywane (przyleciały)
        for task in tasks:
            if task.enter <= time and task.remaining_time > 0:
                ready_to_be_executed.append(task)
                
        # Posortuj 
        num_task_to_exec = sorted(ready_to_be_executed, key=lambda x: (x.duration, x.enter))[0].number
        idx = None
        
        for task_idx, task in enumerate(tasks):
            if task.number == num_task_to_exec:
                tasks[task_idx].being_executed = True
                idx = task_idx
                
        while tasks[idx].remaining_time > 0:
            print(str(time) + " | " + str(tasks[idx].number))
            tasks[idx].execute()
            
            time += 1
            
            # Inkrementuj liczniki czekania zadań
            for task in tasks:
                if not task.being_executed and task.enter < time:
                    wait_time[task.number - 1] += 1
        
        del tasks[idx]
        
    return wait_time

# SJF z wywłaszczeniem
def SJF_preemptive(list_of_processes):
    
    time = 0
    
    # Ustaw procesy wg czasu przyjścia
    tasks = sorted(list_of_processes, key=lambda x: x.enter)
    wait_time = [0] * len(tasks)
    
    # Wykonuj pętlę, dopóki są zadania do wykonania
    while len(tasks) > 0:
        ready_to_be_executed = []

        # Stwórz listę zadań, które mogą być już wykonywane (przyleciały)
        for task in tasks:
            if task.enter <= time and task.remaining_time > 0:
                ready_to_be_executed.append(task)
                
        # Posortuj 
        num_task_to_exec = sorted(ready_to_be_executed, key=lambda x: (x.duration, x.enter))[0].number
        idx = None
        
        for task_idx, task in enumerate(tasks):
            tasks[task_idx].being_executed = False
            
            if task.number == num_task_to_exec:
                tasks[task_idx].being_executed = True
                idx = task_idx
        
        print(str(time) + " | " + str(tasks[idx].number))
        tasks[idx].execute()

        # Inkrementuj liczniki czekania zadań i usuń wykonane zadania
        for task_idx, task in enumerate(tasks):
            if not task.being_executed and task.enter < time:
                wait_time[task.number - 1] += 1
                
            if task.remaining_time == 0:
                del tasks[task_idx]
              
        time += 1
                

    return wait_time

In [464]:
SJF_nonpreemptive(tasks)

0 | 1
1 | 1
2 | 1
3 | 1
4 | 1
5 | 1
6 | 1
7 | 3
8 | 2
9 | 2
10 | 2
11 | 2
12 | 4
13 | 4
14 | 4
15 | 4


[0, 6, 3, 7]

In [314]:
result[3].wait_time

0

In [328]:
tasks = sorted(tasks, key=lambda x: (x.duration, x.enter))

In [332]:
for task in tasks:
    print(str(task.duration) + " | " + str(task.enter) + " | " + str(task.number))

2 | 12 | 5
3 | 5 | 3
4 | 3 | 2
4 | 12 | 4
5 | 0 | 1
